In [50]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler



In [51]:
books_df = pd.read_csv("../data/Books.csv")
users_df = pd.read_csv("../data/Users.csv")
ratings_df = pd.read_csv("../data/ratings.csv")

C:\Users\NelsonWang\AppData\Local\Temp\ipykernel_5804\3971019450.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books_df = pd.read_csv("../data/Books.csv")


In [52]:
ratings_df.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [53]:
# Clean up
# 1. Find the user ID of those who have rater either too few books (<5) and too many books (>200)
# 2. Remove those users from the ratings_df

valid_transaction_df = ratings_df.groupby('User-ID').filter(lambda x: len(x) > 5 and len(x) < 200)
valid_transaction_df.groupby('User-ID').size().sort_values(ascending=False)

User-ID
240403    199
203017    199
193458    199
2033      198
267061    198
         ... 
95420       6
95156       6
231313      6
95146       6
233397      6
Length: 18812, dtype: int64

In [54]:
# Split tests and training by separating the unique users.
unique_users = valid_transaction_df['User-ID'].unique()
train_users, test_users = train_test_split(unique_users, test_size=0.2, random_state=42)

print(len(train_users), len(test_users))
# Split the data
train_df = valid_transaction_df[valid_transaction_df['User-ID'].isin(train_users)]
test_df = valid_transaction_df[valid_transaction_df['User-ID'].isin(test_users)]

print(train_df.shape, test_df.shape)
train_df.head()

15049 3763
(386498, 3) (99399, 3)


,User-ID,ISBN,Book-Rating
16,276747,0060517794,9
17,276747,0451192001,0
18,276747,0609801279,0
19,276747,0671537458,9
20,276747,0679776818,8


In [55]:
#Import the required classes and methods from the surprise library
from surprise import Reader, Dataset, KNNBasic
from surprise.model_selection import cross_validate

# Define the reader object and load the data
reader = Reader(rating_scale=(1, 10))
books_data = Dataset.load_from_df(train_df[['User-ID', 'ISBN', 'Book-Rating']], reader)


In [56]:

knn = KNNBasic()

# Run 5-fold cross-validation and print the results
cross_validate_results = cross_validate(knn, books_data, measures=['RMSE'], cv=5, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    4.2275  4.2216  4.2256  4.2398  4.2344  4.2298  0.0065  
Fit time          6.03    5.57    4.15    4.23    3.14    4.62    1.04    
Test time         3.05    2.88    3.06    2.62    2.23    2.77    0.31    


In [57]:
import pprint
pprint.pprint(cross_validate_results)

{'fit_time': (6.0254762172698975,
              5.567241430282593,
              4.151386499404907,
              4.230288505554199,
              3.1393043994903564),
 'test_rmse': array([4.22751028, 4.22160616, 4.22557362, 4.23976356, 4.23435406]),
 'test_time': (3.046919584274292,
               2.879972457885742,
               3.0570428371429443,
               2.6157963275909424,
               2.226896286010742)}


In [58]:
#Import SVD
from surprise import SVD

#Define the SVD algorithm object
svd = SVD()

#Run 5-fold cross-validation and print the results
cross_validate_results = cross_validate(svd, books_data, measures=['RMSE', 'mae'], cv=5, verbose=True)


KeyboardInterrupt: 

In [ ]:

pprint.pprint(cross_validate_results)

{'fit_time': (5.514612913131714,
              5.796938180923462,
              5.624850034713745,
              5.365435600280762,
              5.229335069656372),
 'test_mae': array([3.15849274, 3.15358235, 3.15846425, 3.14792543, 3.15248702]),
 'test_rmse': array([3.61459375, 3.6069404 , 3.60952643, 3.60024307, 3.60290527]),
 'test_time': (0.5676262378692627,
               0.801414966583252,
               0.5558133125305176,
               0.6515505313873291,
               0.3360731601715088)}


In [ ]:
from surprise import accuracy, Dataset, SVD
from surprise.model_selection import train_test_split

# Load the movielens-100k dataset (download it if needed),
data = Dataset.load_builtin("ml-100k")

# sample random trainset and testset
# test set is made of 25% of the ratings.
trainset, testset = train_test_split(data, test_size=0.25)

# We'll use the famous SVD algorithm.
algo = SVD()

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)

RMSE: 0.9409


0.9408735768722385

In [ ]:
# Trying to predict the rating of a single user for a single book
# We'll use the SVD algorithm

from surprise.model_selection import KFold
from surprise import SVD

kf = KFold(n_splits=5)

algo = SVD()

algo.fit(books_data.build_full_trainset())


In [ ]:

from surprise import Dataset, get_dataset_dir, KNNBaseline

trainset = books_data.build_full_trainset()
# sim_options = {'name': 'cosine', 'user_based': False}
# algo = KNNBaseline(sim_options=sim_options)
algo = KNNBasic()
algo.fit(trainset)


Computing the msd similarity matrix...
Done computing similarity matrix.


In [ ]:
print(algo.trainset.all_items())

range(0, 164715)


In [ ]:
# algo.get_neighbors('1558744290', k=10)
# Find inner id for 1558744290

innid = 20
print(innid)
res = algo.get_neighbors(innid, k=10)


11


In [65]:
def inner_id_to_book_desc(inner_id, trainset):
    raw_id = trainset.to_raw_iid(inner_id)
    book = books_df[books_df['ISBN'] == raw_id]
    return book


def inner_ids_to_book_desc(inner_ids, trainset):
    # Return as dataframe.
    return pd.concat([inner_id_to_book_desc(inner_id, trainset) for inner_id in inner_ids])

def raw_id_to_inner_id(raw_id, trainset):
    return trainset.to_inner_iid(raw_id)

In [78]:
# algo.get_neighbors('1558744290', k=10)
# Find inner id for 1558744290

innid = raw_id_to_inner_id('0439064872', trainset)
print(innid)
res = algo.get_neighbors(innid, k=10)


2045


In [79]:
inner_id_to_book_desc(innid, trainset)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
3459,0439064872,Harry Potter and the Chamber of Secrets (Book 2),J. K. Rowling,2000,Scholastic,http://images.amazon.com/images/P/0439064872.0...,http://images.amazon.com/images/P/0439064872.0...,http://images.amazon.com/images/P/0439064872.0...


In [80]:
inner_ids_to_book_desc(res, trainset)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
14438,3404921178,Nur der Tod ist ohne Makel.,Ann Granger,2002,LÃ?Â¼bbe,http://images.amazon.com/images/P/3404921178.0...,http://images.amazon.com/images/P/3404921178.0...,http://images.amazon.com/images/P/3404921178.0...
101109,0590437186,Stacey's Mistake (Baby-Sitters Club (Paperback)),Ann M. Martin,1988,Scholastic Paperbacks,http://images.amazon.com/images/P/0590437186.0...,http://images.amazon.com/images/P/0590437186.0...,http://images.amazon.com/images/P/0590437186.0...
127862,0743463390,Revenge: A Story of Hope,Laura Blumenfeld,2003,Washington Square Press,http://images.amazon.com/images/P/0743463390.0...,http://images.amazon.com/images/P/0743463390.0...,http://images.amazon.com/images/P/0743463390.0...
155250,1564586812,Look Inside Cross-Sections: Cars,Alan Austin,1994,Dorling Kindersley,http://images.amazon.com/images/P/1564586812.0...,http://images.amazon.com/images/P/1564586812.0...,http://images.amazon.com/images/P/1564586812.0...
248640,0525940774,The Silver Cloud Cafe,Alfredo Vea,1996,Dutton Books,http://images.amazon.com/images/P/0525940774.0...,http://images.amazon.com/images/P/0525940774.0...,http://images.amazon.com/images/P/0525940774.0...
115823,0425163393,Kat Scratch Fever (Kat Colorado Mysteries),Karen Kijewski,1998,Berkley Publishing Group,http://images.amazon.com/images/P/0425163393.0...,http://images.amazon.com/images/P/0425163393.0...,http://images.amazon.com/images/P/0425163393.0...
11098,0425092917,The Accidental Tourist,Anne Tyler,1994,Berkley Publishing Group,http://images.amazon.com/images/P/0425092917.0...,http://images.amazon.com/images/P/0425092917.0...,http://images.amazon.com/images/P/0425092917.0...


In [49]:
books_df.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...
